# Scraping de Natural by Lila

Nuestro objetivo consiste en escrapear la pagina de vestidos de natural by Lila por categorías, por lo que usaremos tanto Selenium como Beautiful Soup.

In [1]:
import sys
import os
import pandas as pd
import json

sys.path.append(os.path.abspath('../'))
from src import funciones_scraping as fs

In [8]:
lista_vestidos_cortos_largos = fs.selenium_bylila()

In [14]:
with open("../datos/scraping/links_nbl.json", "w") as archivo_json:
    json.dump(lista_vestidos_cortos_largos, archivo_json)

### Procedemos a llevar a cabo la extracción de los datos de nuestro interés.

In [192]:
dic_vestido = {
        "nombre" : [],
        "marca" : [],
        "precio" : [],
        "talla" : [],
        "categoria" :[]
        }
i=0
for links_vestidos in lista_vestidos_cortos_largos:
    for url in links_vestidos:
        res = requests.get(url)
        if res.status_code != 200:
            print(url)

        sopa_vestido = BeautifulSoup(res.content, "html.parser")
        
        
        nombre = sopa_vestido.find("span", {"class":"titulosinglegrande"}).getText()
        marca = "Natural by Lila"
        precio = float(sopa_vestido.find("span", {"class":"woocommerce-Price-amount amount"}).getText().replace("€", ""))

        # Sacamos las tallas
        desc = sopa_vestido.find("div", {"class":"content-desc"}).findAll("p")[1].text.lower()
        table = sopa_vestido.find("table", {"class":"variations"})

        if "talla única" in desc:
            lista_tallas = ["S", "M"]

        elif "disponible en talla" in desc:
            tallas_disp = desc.split("talla ")[1:]
            lista_tallas=[]
            for talla in tallas_disp:
                lista_tallas.append(talla[0].upper())
                
        elif len(table.findAll('tr')) > 3:
            fila_tallas = table.findAll("tr")[3]
            tallas = fila_tallas.findAll("label")
            lista_tallas=[]
            for talla in tallas:
                tallas_separadas = talla.text.upper().split("/")
                lista_tallas.extend(tallas_separadas)
                lista_tallas= list(set(lista_tallas))
                    
        else:
            lista_tallas=[sopa_vestido.find("div", {"class":"model-info"}).text[-1].upper()]

        # Creamos los diccionarios:
        for talla in lista_tallas:
            dic_vestido["nombre"].append(nombre)
            dic_vestido["marca"].append(marca)
            dic_vestido["precio"].append(precio)
            dic_vestido["talla"].append(talla)
            if i == 0:
                dic_vestido["categoria"].append("corto")
            else:
                dic_vestido["categoria"].append("largo")
    i+=1


In [3]:
with open("../datos/scraping/links_nbl.json", "r") as archivo_json:
    lista_vestidos_cortos_largos = json.load(archivo_json)

In [7]:
dic_vestido = fs.extraccion_info(lista_vestidos_cortos_largos)

In [8]:
df_vestidos_mbl = pd.DataFrame(dic_vestido)
df_vestidos_mbl

,nombre,marca,precio,talla,categoria
0,Mini Vestido Strapless Cascada,Natural by Lila,49.99,S,corto
1,Mini Vestido Strapless Cascada,Natural by Lila,49.99,M,corto
2,Vestido Asimétrico Choker Lurex,Natural by Lila,39.99,S,corto
3,Vestido Asimétrico Choker Lurex,Natural by Lila,39.99,M,corto
4,Vestido Básico Bella,Natural by Lila,36.99,S,corto
...,...,...,...,...,...
338,Vestido Mimi,Natural by Lila,39.95,M,largo
339,Vestido Anudado Manga Larga,Natural by Lila,34.95,S,largo
340,Vestido Anudado Manga Larga,Natural by Lila,34.95,M,largo
341,Vestido Halter Largo,Natural by Lila,39.95,S,largo


In [132]:
df_cortos_mbl.shape

(84, 5)

In [ ]:
# i=1
# dic_vestido = {
#         "nombre" : [],
#         "marca" : [],
#         "precio" : [],
#         "talla" : [],
#         "categoria" :[]
#         }

# for url in links_vestidos_largos:
#     res = requests.get(url)
#     if res.status_code != 200:
#         print(url)

#     sopa_vestido = BeautifulSoup(res.content, "html.parser")
    
    
#     nombre = sopa_vestido.find("span", {"class":"titulosinglegrande"}).getText()
#     marca = "Natural by Lila"
#     precio = float(sopa_vestido.find("span", {"class":"woocommerce-Price-amount amount"}).getText().replace("€", ""))

#     # Sacamos las tallas
#     desc = sopa_vestido.find("div", {"class":"content-desc"}).findAll("p")[1].text.lower()
#     table = sopa_vestido.find("table", {"class":"variations"})

#     if "talla única" in desc:
#         lista_tallas = ["S", "M"]

#     elif "disponible en talla" in desc:
#         tallas_disp = desc.split("talla ")[1:]
#         lista_tallas=[]
#         for talla in tallas_disp:
#             lista_tallas.append(talla[0].upper())

#     elif len(table.findAll('tr')) > 3:
#         fila_tallas = table.findAll("tr")[3]
#         tallas = fila_tallas.findAll("label")
#         lista_tallas=[]
#         for talla in tallas:
#             tallas_separadas = talla.text.upper().split("/")
#             lista_tallas.extend(tallas_separadas)
#             lista_tallas= list(set(lista_tallas))
    
#     else:
#         lista_tallas=[sopa_vestido.find("div", {"class":"model-info"}).text[-1].upper()]

#     # Creamos los diccionarios:
#     for talla in lista_tallas:
#         dic_vestido["nombre"].append(nombre)
#         dic_vestido["marca"].append(marca)
#         dic_vestido["precio"].append(precio)
#         dic_vestido["talla"].append(talla)
#         if i == 0:
#             dic_vestido["categoria"].append("corto")
#         else:
#             dic_vestido["categoria"].append("largo")

https://naturalbylila.com/vestido-largo-lurex-asimetrico/
https://naturalbylila.com/vestido-cutout-drapeado-lurex/
https://naturalbylila.com/vestido-midi-cenido-leopardo/
https://naturalbylila.com/vestido-boho-pedreria/
https://naturalbylila.com/vestido-largo-cascada-lurex/
https://naturalbylila.com/vestido-espalda-cruzada/
https://naturalbylila.com/vestido-bertha/


In [181]:
len(links_vestidos_largos)

128

In [186]:
df_largos_mbl = pd.DataFrame(dic_vestido)
df_largos_mbl["talla"].value_counts()

talla
S     128
M     121
L       9
XS      1
Name: count, dtype: int64

In [191]:
df_cortos_mbl.shape

(84, 5)

In [196]:
df_vestidos_mbl["talla"].isna().sum()

np.int64(0)

In [197]:
df_vestidos_mbl

,nombre,marca,precio,talla,categoria
0,Mini Vestido Strapless Cascada,Natural by Lila,49.99,S,corto
1,Mini Vestido Strapless Cascada,Natural by Lila,49.99,M,corto
2,Vestido Asimétrico Choker Lurex,Natural by Lila,39.99,S,corto
3,Vestido Asimétrico Choker Lurex,Natural by Lila,39.99,M,corto
4,Vestido Básico Bella,Natural by Lila,36.99,S,corto
...,...,...,...,...,...
338,Vestido Mimi,Natural by Lila,39.95,M,largo
339,Vestido Anudado Manga Larga,Natural by Lila,34.95,S,largo
340,Vestido Anudado Manga Larga,Natural by Lila,34.95,M,largo
341,Vestido Halter Largo,Natural by Lila,39.95,S,largo
